In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import logging
logging.basicConfig(level=logging.ERROR)
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [2]:
!pip install transformers==2.8.0

     |████████████████████████████████| 573kB 5.6MB/s 
     |████████████████████████████████| 133kB 34.5MB/s 
     |████████████████████████████████| 890kB 24.9MB/s 
     |████████████████████████████████| 5.6MB 30.9MB/s 
     |████████████████████████████████| 1.2MB 40.1MB/s 
     |████████████████████████████████| 7.3MB 37.3MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=c5821bd28c5a340054c7a308d336288277c7b3f8035e8c5f32149ffe2d0ceb2e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.20.35 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [3]:
!pip install tensorflow==1.13.1

     |████████████████████████████████| 92.6MB 48kB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 368kB 42.9MB/s 
     |████████████████████████████████| 3.2MB 38.9MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp -r "/content/drive/MyDrive/Information_retrieval_project/model_save_categorized_reduced_QC" /content

In [6]:
!cp "/content/drive/MyDrive/Information_retrieval_project/QC_science/train_QC_data.csv" /content
!cp "/content/drive/MyDrive/Information_retrieval_project/QC_science/test_QC_data.csv" /content
!cp "/content/drive/MyDrive/Information_retrieval_project/QC_science/val_QC_data.csv" /content


In [7]:
import pandas as pd
train_data = pd.read_csv("train_QC_data.csv")
test_data = pd.read_csv("test_QC_data.csv")
val_data = pd.read_csv("val_QC_data.csv")
train_data


,questionID,originalQuestionID,totalPossiblePoint,AnswerKey,isMultipleChoiceQuestion,includesDiagram,examName,grade,year,QCLabel,Question,subject,category,fold
0,VASoL_2008_3_34,34,1,C,1,0,Virginia Standards of Learning - Science,3,2008,matter_properties of objects_TEXT,A student is asked to bring something that fee...,NaN,Train,Easy
1,MCAS_2015_8_6,6,1,B,1,0,MCAS,8,2015,celestial_FEATURES_STELLAR,Which of the following statements best describ...,NaN,Test,Easy
2,Mercury_SC_417677,417677,1,B,1,0,Mercury,4,2015,energy_LIGHT_REFLECT,A polished metal ball looks very shiny and bri...,NaN,Test,Challenge
3,Mercury_7230423,7230423,1,A,1,0,Mercury,9,2015,LIFE_EXTINCTION_MASSEX,Which was a main force driving extensive speci...,NaN,Test,Easy
4,NYSEDREGENTS_2007_8_6,6,1,2,1,0,NYSEDREGENTS,8,2007,Life_functions_features and functions_CELLBIO_...,Compared to the amount of hereditary informati...,NaN,Train,Challenge
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5592,Mercury_402502,402502,1,D,1,0,Mercury,8,2015,matter_chemistry_periodic table,"According to the periodic table, argon is foun...",NaN,Test,Challenge
5593,MCAS_2006_9_20,20,1,B,1,0,MCAS,9,2006,FOR_MOMENTUM,Which of the following has the least momentum?...,NaN,Train,Challenge
5594,NYSEDREGENTS_2013_8_35,35,1,4,1,0,NYSEDREGENTS,8,2013,Life_functions_features and functions_PLANT_PH...,The amount of which greenhouse gas in the air ...,NaN,Test,Easy
5595,Mercury_7082670,7082670,1,C,1,0,Mercury,7,2015,energy_LIGHT_electromagnetic spectrum,The visible light spectrum can be subdivided a...,NaN,Test,Easy


In [8]:
from google.colab import files

In [9]:
import re
def clean_sentence(question):
  # print(question)
  question = re.sub('<[^>]*>', ' ',question)
  question = re.sub(' +', ' ', question)
  question = re.sub('\xa0','',question)
  question = question.rstrip()
  question = re.sub('nan','',question)
  question = re.sub(u'\u2004','',question)
  question = re.sub(u'\u2009','',question)

  # question = question.decode("utf-8")
  # question = question.replace(u'\u200\d*','').encode("utf-8")
  question = re.sub('&nbsp','',question)
  question = re.sub('&ndash','',question)
  question = re.sub('\r','',question)
  question = re.sub('\t','',question)
  question = re.sub('\n',' ',question)

  question = re.sub('MathType@.*','',question)
  question = re.sub('&thinsp','',question)
  question = re.sub('&times','',question)
  question = re.sub('\u200b','',question)
  question = re.sub('&rarr;;;','',question)

  return question

In [10]:
train_data["QCLabel"].value_counts()

celestial_cycles                           106
matter_chemistry_periodic table             85
matter_chemistry_atomic                     79
matter_CHANGES_CHEMICAL                     72
science_INFERENCE_observation               69
                                          ... 
EARTH_human impacts_WHAT_pollution_SOIL      1
energy_LIGHT_SPEED                           1
matter_measurement_UNIT_PH                   1
EARTH_human impacts_RES_MINING               1
science_INFERENCE_INFERENCE                  1
Name: QCLabel, Length: 416, dtype: int64

In [11]:
# final_data_1 = final_data.loc[0:71003,:]
# final_data_1

In [12]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Loading BERT tokenizer...


In [13]:
test_data["QCLabel"].value_counts()

Life_reproduction_DNA inheritance_inheritance             26
celestial_cycles                                          26
Life_functions_features and functions_PLANT_PHOTOSYNTH    22
matter_chemistry_atomic                                   21
science_INFERENCE_experiment design                       19
                                                          ..
EARTH_INNER_internal heat                                  1
EARTH_SOIL_ecosystem                                       1
matter_ENVEFF_LIGHT                                        1
EARTH_GEO_FORMATIONS_EARTH_INNER_generic plate             1
LIFE_environment and adaptation_ADAP_animal                1
Name: QCLabel, Length: 352, dtype: int64

In [14]:

from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
LE.fit_transform(pd.concat([train_data['QCLabel'],test_data['QCLabel']]))
train_data['label'] = LE.transform(train_data['QCLabel'])
train_data.head()

,questionID,originalQuestionID,totalPossiblePoint,AnswerKey,isMultipleChoiceQuestion,includesDiagram,examName,grade,year,QCLabel,Question,subject,category,fold,label
0,VASoL_2008_3_34,34,1,C,1,0,Virginia Standards of Learning - Science,3,2008,matter_properties of objects_TEXT,A student is asked to bring something that fee...,NaN,Train,Easy,399
1,MCAS_2015_8_6,6,1,B,1,0,MCAS,8,2015,celestial_FEATURES_STELLAR,Which of the following statements best describ...,NaN,Test,Easy,260
2,Mercury_SC_417677,417677,1,B,1,0,Mercury,4,2015,energy_LIGHT_REFLECT,A polished metal ball looks very shiny and bri...,NaN,Test,Challenge,293
3,Mercury_7230423,7230423,1,A,1,0,Mercury,9,2015,LIFE_EXTINCTION_MASSEX,Which was a main force driving extensive speci...,NaN,Test,Easy,103
4,NYSEDREGENTS_2007_8_6,6,1,2,1,0,NYSEDREGENTS,8,2007,Life_functions_features and functions_CELLBIO_...,Compared to the amount of hereditary informati...,NaN,Train,Challenge,171


In [15]:
def get_labels(prediction):
    predicted_label =  LE.inverse_transform([prediction])
    return predicted_label[0]

In [16]:
get_labels(204)

'Life_functions_features and functions_PLANT_VASCULAR'

In [17]:
train_data.iloc[14,1]

'29'

In [18]:
train_data

,questionID,originalQuestionID,totalPossiblePoint,AnswerKey,isMultipleChoiceQuestion,includesDiagram,examName,grade,year,QCLabel,Question,subject,category,fold,label
0,VASoL_2008_3_34,34,1,C,1,0,Virginia Standards of Learning - Science,3,2008,matter_properties of objects_TEXT,A student is asked to bring something that fee...,NaN,Train,Easy,399
1,MCAS_2015_8_6,6,1,B,1,0,MCAS,8,2015,celestial_FEATURES_STELLAR,Which of the following statements best describ...,NaN,Test,Easy,260
2,Mercury_SC_417677,417677,1,B,1,0,Mercury,4,2015,energy_LIGHT_REFLECT,A polished metal ball looks very shiny and bri...,NaN,Test,Challenge,293
3,Mercury_7230423,7230423,1,A,1,0,Mercury,9,2015,LIFE_EXTINCTION_MASSEX,Which was a main force driving extensive speci...,NaN,Test,Easy,103
4,NYSEDREGENTS_2007_8_6,6,1,2,1,0,NYSEDREGENTS,8,2007,Life_functions_features and functions_CELLBIO_...,Compared to the amount of hereditary informati...,NaN,Train,Challenge,171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5592,Mercury_402502,402502,1,D,1,0,Mercury,8,2015,matter_chemistry_periodic table,"According to the periodic table, argon is foun...",NaN,Test,Challenge,357
5593,MCAS_2006_9_20,20,1,B,1,0,MCAS,9,2006,FOR_MOMENTUM,Which of the following has the least momentum?...,NaN,Train,Challenge,95
5594,NYSEDREGENTS_2013_8_35,35,1,4,1,0,NYSEDREGENTS,8,2013,Life_functions_features and functions_PLANT_PH...,The amount of which greenhouse gas in the air ...,NaN,Test,Easy,198
5595,Mercury_7082670,7082670,1,C,1,0,Mercury,7,2015,energy_LIGHT_electromagnetic spectrum,The visible light spectrum can be subdivided a...,NaN,Test,Easy,297


In [19]:
# LE_test = LabelEncoder()

test_data['label'] = LE.transform(test_data['QCLabel'])
test_data.head()

,questionID,originalQuestionID,totalPossiblePoint,AnswerKey,isMultipleChoiceQuestion,includesDiagram,examName,grade,year,QCLabel,Question,subject,category,fold,label
0,Mercury_409529,409529,1,B,1,0,Mercury,7,2015,science_INFERENCE_experiment design,Robert is a fisherman who wants to find a way ...,NaN,Dev,Challenge,416
1,Mercury_7090790,7090790,1,A,1,0,Mercury,7,2015,matter_Change of state_EVAPoration,Which of these factors causes water to evapora...,NaN,Test,Challenge,336
2,TIMSS_2007_8_pg7,pg7,1,C,1,0,TIMSS,8,2007,matter_chemistry_atomic,Which statement is true about the particles of...,NaN,Train,Challenge,354
3,Mercury_7014455,7014455,1,A,1,0,Mercury,8,2015,energy_LIGHT_GENERICPROP,Which generates waves that are capable of trav...,NaN,Dev,Easy,292
4,NAEP_2000_8_S21+4,S21+4,1,C,1,0,NAEP,8,2000,forces and friction,To keep a heavy box sliding across a carpeted ...,NaN,Test,Challenge,324


In [20]:
val_data['label'] = LE.transform(val_data['QCLabel'])
val_data.head()

,questionID,originalQuestionID,totalPossiblePoint,AnswerKey,isMultipleChoiceQuestion,includesDiagram,examName,grade,year,QCLabel,Question,subject,category,fold,label
0,MCAS_2006_9_30-v1,30,1,D,1,0,MCAS,9,2006,matter_CHANGES_PHYSICAL,Which of the following changes occurs as a sol...,NaN,Train,Challenge,329
1,Mercury_SC_401144,401144,1,B,1,0,Mercury,5,2015,EARTH_WEATHER_CLOUDS,"When water vapor rises and cools, the liquid w...",NaN,Train,Easy,43
2,NCEOGA_2013_8_46,46,1,A,1,0,North Carolina READY End-of-Grade Assessment,8,2013,EARTH_GEO_FORMATIONS,Which best describes the characteristics of a ...,NaN,Dev,Easy,2
3,Mercury_417146,417146,1,A,1,0,Mercury,8,2015,Life_interdependence_ecological features,Most of the oxygen in the atmosphere is made b...,NaN,Dev,Challenge,222
4,Mercury_7283833,7283833,1,B,1,0,Mercury,8,2015,LIFE_HEALTH_DIESEASE_PANDEMICEPIDEMIC,Which aspect of modern Life_could most likely ...,NaN,Test,Challenge,108


In [21]:
train_features, test_features, train_labels, test_labels = train_data["Question"],test_data["Question"],train_data["label"],test_data["label"]
val_features,val_labels = val_data["Question"], val_data["label"]

In [22]:
train_labels.value_counts()

268    106
357     85
354     79
328     72
417     69
      ... 
320      1
91       1
88       1
363      1
336      1
Name: label, Length: 416, dtype: int64

In [23]:
test_labels.value_counts()

268    26
231    26
198    22
354    21
416    19
       ..
291     1
128     1
285     1
283     1
311     1
Name: label, Length: 352, dtype: int64

In [24]:
get_labels(268)

'celestial_cycles'

In [25]:
question_answer = train_features.values
categories = train_labels.values

In [26]:
question_answer

array(['A student is asked to bring something that feels rough to class. Which would be BEST for him to bring? (A) Pillow (B) Marble (C) Sandpaper (D) Trading card',
       'Which of the following statements best describes the role of gravity in the formation of stars? (A) Gravity converts solid matter into gases and light energy. (B) Gravity causes gases and dust particles to condense into spheres. (C) Gravity cools gases and liquids until they become one solid mass. (D) Gravity pushes rocks and dust particles outward from a dense center.',
       'A polished metal ball looks very shiny and bright on a sunny day. What makes the ball look shiny? (A) The ball makes light. (B) The ball reflects light. (C) The ball absorbs light and then releases it. (D) The ball absorbs light and keeps it inside.',
       ...,
       'The amount of which greenhouse gas in the air will increase the most if large forests are cut down to be used for building materials without planting new trees in their pla

In [27]:
len(categories)

5597

In [28]:
input_ids = []
attention_masks = []

for sent in question_answer:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)


# Print sentence 0, now as a list of IDs.
print('Original: ', question_answer[0])
print('Token IDs:', input_ids[0])

Original:  A student is asked to bring something that feels rough to class. Which would be BEST for him to bring? (A) Pillow (B) Marble (C) Sandpaper (D) Trading card
Token IDs: tensor([  101,  1037,  3076,  2003,  2356,  2000,  3288,  2242,  2008,  5683,
         5931,  2000,  2465,  1012,  2029,  2052,  2022,  2190,  2005,  2032,
         2000,  3288,  1029,  1006,  1037,  1007, 10005,  1006,  1038,  1007,
         7720,  1006,  1039,  1007,  5472, 23298,  1006,  1040,  1007,  6202,
         4003,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,

In [29]:
input_ids_val = []
attention_masks_val = []

for sent in val_features:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids_val.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_val.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids_val = torch.cat(input_ids_val, dim=0)
attention_masks_val = torch.cat(attention_masks_val, dim=0)


# Print sentence 0, now as a list of IDs.
print('Original: ', question_answer[0])
print('Token IDs:', input_ids[0])

Original:  A student is asked to bring something that feels rough to class. Which would be BEST for him to bring? (A) Pillow (B) Marble (C) Sandpaper (D) Trading card
Token IDs: tensor([  101,  1037,  3076,  2003,  2356,  2000,  3288,  2242,  2008,  5683,
         5931,  2000,  2465,  1012,  2029,  2052,  2022,  2190,  2005,  2032,
         2000,  3288,  1029,  1006,  1037,  1007, 10005,  1006,  1038,  1007,
         7720,  1006,  1039,  1007,  5472, 23298,  1006,  1040,  1007,  6202,
         4003,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,

In [30]:
print('Original: ', question_answer[1])
print('Token IDs:', input_ids[1])

Original:  Which of the following statements best describes the role of gravity in the formation of stars? (A) Gravity converts solid matter into gases and light energy. (B) Gravity causes gases and dust particles to condense into spheres. (C) Gravity cools gases and liquids until they become one solid mass. (D) Gravity pushes rocks and dust particles outward from a dense center.
Token IDs: tensor([  101,  2029,  1997,  1996,  2206,  8635,  2190,  5577,  1996,  2535,
         1997,  8992,  1999,  1996,  4195,  1997,  3340,  1029,  1006,  1037,
         1007,  8992, 19884,  5024,  3043,  2046, 15865,  1998,  2422,  2943,
         1012,  1006,  1038,  1007,  8992,  5320, 15865,  1998,  6497,  9309,
         2000, 24707, 12325,  2046, 19885,  1012,  1006,  1039,  1007,  8992,
         4658,  2015, 15865,  1998, 26820,  2127,  2027,  2468,  2028,  5024,
         3742,  1012,  1006,  1040,  1007,  8992, 13956,  5749,  1998,  6497,
         9309, 15436,  2013,  1037,  9742,  2415,  1012,   1

In [31]:
labels = torch.tensor(categories)

In [32]:
get_labels(419)

'science_SOURCES'

In [33]:
get_labels(311)

'energy_WAVES_types of motion'

In [34]:
num_classes = len(list(set(categories)))
num_classes

416

In [35]:
from torch.utils.data import TensorDataset, random_split
# train_poincare_tensor = torch.tensor(poincare_embeddings_final,dtype=torch.float)
# train_poincare_tensor = torch.tensor(poincare_embeddings_final_train,dtype=torch.float)
# val_poincare_tensor = torch.tensor(poincare_embeddings_final_val, dtype=torch.float)
train_labels = torch.tensor(categories)
val_labels = torch.tensor(val_labels.values)
# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(input_ids, attention_masks, train_labels)
val_dataset = TensorDataset(input_ids_val,attention_masks_val,val_labels)


In [36]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


In [37]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 32
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size 
        )

In [38]:

# run this cell to prepare model for inference
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Loads BertForSequenceClassification, the pretrained BERT model with a single 
model = BertForSequenceClassification.from_pretrained(
    "model_save_categorized_reduced_QC", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 420,   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = True, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [39]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [40]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [41]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [42]:
test_features = test_features.values
labels = test_labels.values

In [43]:
test_features

array(["Robert is a fisherman who wants to find a way to catch more fish. He decided to try different sizes of hooks. Robert caught 4 catfish, 3 trout, and 7 perch while using worms for bait. Which is the independent (manipulated) variable in Robert's investigation? (A) type of bait (B) size of hook (C) type of fish caught (D) number of fish caught",
       'Which of these factors causes water to evaporate the fastest? (A) high temperatures (B) high humidity (C) slow winds (D) slow runoff',
       'Which statement is true about the particles of a liquid compared to the particles of a gas? (A) Particles of a liquid are a slower and further apart. (B) Particles of a liquid are faster and farther apart. (C) Particles of a liquid are slower and closer together. (D) Particles of a liquid are faster and closer together.',
       ...,
       'Which of these is not an inherited trait in humans? (A) height (B) hair color (C) skin color (D) intelligence',
       'In order to survive, all animals

In [44]:
train_labels = train_data["QCLabel"].values
len(train_labels)

5597

In [45]:
len(input_ids)

5597

In [46]:
# train_embeddings = []
# with torch.no_grad():
#   outputs = model(input_ids.to(device),attention_masks.to(device))
# train_embeddings = torch.mean(outputs[1][0].squeeze(),dim=1)
# train_embeddings.shape

In [47]:
import numpy as np
class_emb = {cls:[] for cls in list(set(train_data["QCLabel"].values))}
for index,label in enumerate(list(set(train_data["QCLabel"].values))):
  sample_indices_for_label = np.where(train_labels == label)[0][:2]
  input_ids_for_class = input_ids[sample_indices_for_label]
  attention_masks_class = attention_masks[sample_indices_for_label]
  input_ids_for_class = input_ids_for_class.to(device)
  attention_masks_class = attention_masks_class.to(device)
  
  with torch.no_grad():
    outputs = model(input_ids_for_class,attention_masks_class)
  class_emb[label] = torch.cat((outputs[1][-3][0][0],outputs[1][-4][0][0],outputs[1][-6][0][0]),dim=-1)


In [48]:
class_keys = [item[0] for item in class_emb.items()]

In [49]:
class_values = [item[1] for item in class_emb.items()]

In [50]:
class_keys = np.array(class_keys)

In [51]:
class_prototype_embeddings = torch.stack(class_values,dim=0)

In [52]:
class_prototype_embeddings.shape

torch.Size([416, 2304])

In [53]:
class_keys[0]

'EARTH_WEATHER_WATER cycle_PREC_EARTH_WEATHER_WATER cycle_EVAPCOND'

In [54]:
len(input_ids)

5597

In [55]:
test_features

array(["Robert is a fisherman who wants to find a way to catch more fish. He decided to try different sizes of hooks. Robert caught 4 catfish, 3 trout, and 7 perch while using worms for bait. Which is the independent (manipulated) variable in Robert's investigation? (A) type of bait (B) size of hook (C) type of fish caught (D) number of fish caught",
       'Which of these factors causes water to evaporate the fastest? (A) high temperatures (B) high humidity (C) slow winds (D) slow runoff',
       'Which statement is true about the particles of a liquid compared to the particles of a gas? (A) Particles of a liquid are a slower and further apart. (B) Particles of a liquid are faster and farther apart. (C) Particles of a liquid are slower and closer together. (D) Particles of a liquid are faster and closer together.',
       ...,
       'Which of these is not an inherited trait in humans? (A) height (B) hair color (C) skin color (D) intelligence',
       'In order to survive, all animals

In [56]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
test_input_ids = []
test_attention_masks = []
for sent in test_features:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    test_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    test_attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)
labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 32  
# test_poincare_tensor = torch.tensor(taxonomy_vectors,dtype=torch.float)

# Create the DataLoader.
prediction_data = TensorDataset(test_input_ids, test_attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [57]:
test_poincare_tensor = class_prototype_embeddings# torch.tensor(taxonomy_vectors,dtype=torch.float)


In [58]:
test_labels = np.array(test_labels)

In [59]:
test_labels[0]

416

In [60]:
len(input_ids)

5597

In [79]:
print('Predicting labels for {:,} test sentences...'.format(len(test_input_ids)))

# Put model in evaluation mode
model.eval()
cos = torch.nn.CosineSimilarity(dim=-1, eps=1e-6)

test_input_ids = test_input_ids.to('cuda')
test_attention_masks = test_attention_masks.to('cuda')
class_prototype_embeddings = class_prototype_embeddings.to('cuda')
# Tracking variables1
predictions , true_labels = [], []
for input_id,attention_mask in zip(test_input_ids, test_attention_masks):
  with torch.no_grad():
    outputs = model(input_id.reshape(1,-1),attention_mask.reshape(1,-1))
  # print(torch.mean(outputs[1][0].squeeze(),dim=0).shape)
  distances = cos(torch.cat((outputs[1][-3][0][0], outputs[1][-4][0][0],outputs[1][-6][0][0])),class_prototype_embeddings)
  distances,indices = torch.topk(distances,20,largest=True)
  predictions.append(class_keys[indices.cpu().numpy()])
print(len(predictions))

Predicting labels for 1,400 test sentences...
1400


In [80]:
labels

array([416, 336, 354, ..., 231, 159,  28])

In [81]:
labels=test_data['label'].values

In [82]:
labels

array([416, 336, 354, ..., 231, 159,  28])

In [83]:
final_predictions = []
for prediction in predictions:
  final_predictions.append(LE.transform(prediction))


Following four cells show metrics Recall@5, R@10, R@15, R@20 for prototype inspired baseline code above


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 5)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 5)

tmp_rank = tf.nn.top_k(y_pred, 5)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1400, 5) (1400,)
precision 0.09685714285714286
update_recall:  0.48428571428571426
recall 0.48428571428571426
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 678.0, 722.0, 678.0, 6322.0]
TMP_RANK:  TopKV2(values=array([[416, 415, 414, 403, 323],
       [336, 100,  54,  51,  32],
       [375, 333, 325, 321,  98],
       ...,
       [231, 225, 153, 131, 117],
       [213, 203, 159, 148, 125],
       [251, 102,  13,  10,   8]]), indices=array([[4, 0, 1, 3, 2],
       [0, 4, 3, 2, 1],
       [4, 1, 3, 0, 2],
       ...,
       [0, 1, 4, 2, 3],
       [2, 1, 0, 3, 4],
       [4, 2, 3, 0, 1]], dtype=int32))


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 10)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 10)

tmp_rank = tf.nn.top_k(y_pred, 10)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1400, 10) (1400,)
precision 0.061714285714285715
update_recall:  0.6171428571428571
recall 0.6171428571428571
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 864.0, 536.0, 864.0, 13136.0]
TMP_RANK:  TopKV2(values=array([[416, 415, 414, ..., 146,  91,  64],
       [337, 336, 334, ...,  51,  32,  27],
       [408, 375, 333, ...,  98,  92,  51],
       ...,
       [380, 231, 230, ..., 153, 131, 117],
       [213, 203, 164, ..., 125, 122, 121],
       [257, 251, 102, ...,  10,   9,   8]]), indices=array([[4, 0, 1, ..., 9, 6, 5],
       [7, 0, 5, ..., 2, 1, 8],
       [9, 4, 1, ..., 2, 8, 7],
       ...,
       [9, 0, 8, ..., 4, 2, 3],
       [2, 1, 7, ..., 4, 8, 6],
       [5, 4, 2, ..., 0, 7, 1]], dtype=int32))


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 15)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 15)

tmp_rank = tf.nn.top_k(y_pred, 15)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1400, 15) (1400,)
precision 0.045
update_recall:  0.675
recall 0.675
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 945.0, 455.0, 945.0, 20055.0]
TMP_RANK:  TopKV2(values=array([[416, 415, 414, ...,  91,  64,  42],
       [337, 336, 334, ...,  32,  27,   6],
       [408, 375, 337, ...,  96,  92,  51],
       ...,
       [380, 235, 231, ..., 131, 117, 111],
       [213, 205, 203, ..., 118,  78,  51],
       [257, 256, 254, ...,  10,   9,   8]]), indices=array([[ 4,  0,  1, ...,  6,  5, 14],
       [ 7,  0,  5, ...,  1,  8, 13],
       [ 9,  4, 10, ..., 12,  8,  7],
       ...,
       [ 9, 12,  0, ...,  2,  3, 13],
       [ 2, 11,  1, ..., 10, 12, 14],
       [ 5, 10, 12, ...,  0,  7,  1]], dtype=int32))


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 20)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 20)

tmp_rank = tf.math.top_k(y_pred, 20,sorted=False)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(y_pred))

(1400, 20) (1400,)
precision 0.03617857142857143
update_recall:  0.7235714285714285
recall 0.7235714285714285
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1013.0, 387.0, 1013.0, 26987.0]
TMP_RANK:  [[415 414 323 ...  19  99 411]
 [336  32  51 ...  44  62 104]
 [321 333  98 ... 292 341 309]
 ...
 [231 225 131 ... 105 129 396]
 [159 203 213 ... 163  81 151]
 [ 10   8 102 ...  27 252 245]]


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 20)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 20)

tmp_rank = tf.math.top_k(y_pred, 20,sorted=False)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(y_pred))

(1400, 20) (1400,)
precision 0.03617857142857143
update_recall:  0.7235714285714285
recall 0.7235714285714285
STREAM_VARS:  [1013.0, 387.0, 1013.0, 26987.0]
TMP_RANK:  [[415 414 323 ...  19  99 411]
 [336  32  51 ...  44  62 104]
 [321 333  98 ... 292 341 309]
 ...
 [231 225 131 ... 105 129 396]
 [159 203 213 ... 163  81 151]
 [ 10   8 102 ...  27 252 245]]


In [ ]:
def mrr_metric(labels, predictions, weights=None,
              metrics_collections=None,
              updates_collections=None,
              name=None):
    
    with tf.name_scope(name, 'mrr_metric', [predictions, labels, weights]) as scope:

    
        k = 20 #predictions.get_shape().as_list()[-1]
        print(predictions.get_shape())

        get_ranked_indicies = tf.expand_dims(tf.where(tf.equal(tf.cast(predictions,tf.int64),labels[:,None]))[:,1],1)
        rr = 1/(get_ranked_indicies+1)
        m_rr =  tf.reduce_sum(rr)/tf.cast(labels.get_shape().as_list()[0],dtype=tf.float64)

        if metrics_collections:
            tf.add_to_collection(metrics_collections, m_rr)

        if updates_collections:
            tf.add_to_collections(updates_collections, update_mrr_op)

        return m_rr,m_rr,rr

In [ ]:
mrr, update_mrr,rr = mrr_metric(y_true,y_pred)
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("update_mrr",sess.run(update_mrr),sess.run(rr).shape)
    # print("precision",sess.run(precision))

    # print("update_recall: ",sess.run(update_recall ))
    # print("recall",sess.run(recall))

    # print("STREAM_VARS: ",(sess.run(stream_vars)))
    # print("TMP_RANK: ",sess.run(tmp_rank))

(1400, 10)
update_mrr 0.3498041383219954 (864, 1)


In [86]:
def get_cleaned_taxonomy(taxonomy):
  cleaned_taxonomy = []
  for value in taxonomy:
      value = ' '.join(value.lower().split("_"))
      # taxonomy_words = [inflection.singularize(val)  for token in value for val in token.split(" ") if val.isalpha()]
      cleaned_taxonomy.append( value )
  return cleaned_taxonomy
test_labels = list(set(test_data["QCLabel"].values))
test_emb_data = get_cleaned_taxonomy(test_labels)

In [87]:
label_input_ids = []
label_attention_masks = []
for sent in test_emb_data:

    label_encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    label_input_ids.append(label_encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    label_attention_masks.append(label_encoded_dict['attention_mask'])

In [88]:
import numpy as np
taxonomy_vectors = []
for label_input_id,label_att_mask in zip(label_input_ids,label_attention_masks):
    label_input_id = label_input_id.to(device)
    label_att_mask = label_att_mask.to(device)
    with torch.no_grad():
      outputs = model(label_input_id.reshape(1,-1),label_att_mask.reshape(1,-1))
    taxonomy_vectors.append(torch.cat((outputs[1][-2][0][0], outputs[1][-3][0][0],outputs[1][-4][0][0])).cpu().numpy())
taxonomy_vectors = np.vstack(taxonomy_vectors)
taxonomy_vectors.shape


(352, 2304)

In [89]:
labels

array([416, 336, 354, ..., 231, 159,  28])

In [90]:
test_labels = np.array(test_labels)

In [101]:
print('Predicting labels for {:,} test sentences...'.format(len(test_input_ids)))

# Put model in evaluation mode
model.eval()
cos = torch.nn.CosineSimilarity(dim=-1, eps=1e-6)

test_input_ids = test_input_ids.to('cuda')
test_attention_masks = test_attention_masks.to('cuda')
taxonomy_vectors = torch.tensor(taxonomy_vectors,dtype=torch.float).to('cuda')
# Tracking variables1
predictions , true_labels = [], []
for input_id,attention_mask in zip(test_input_ids, test_attention_masks):
  with torch.no_grad():
    outputs = model(input_id.reshape(1,-1),attention_mask.reshape(1,-1))
  # print(torch.mean(outputs[1][0].squeeze(),dim=0).shape)
  distances = cos(torch.cat((outputs[1][-2][0][0], outputs[1][-3][0][0],outputs[1][-4][0][0])),taxonomy_vectors)
  distances,indices = torch.topk(distances,5,largest=True)
  predictions.append(test_labels[indices.cpu().numpy()])
print(len(predictions))

Predicting labels for 1,400 test sentences...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


1400


In [102]:
final_predictions = []
for prediction in predictions:
  final_predictions.append(LE.transform(prediction))


In [103]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 5)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 5)

tmp_rank = tf.nn.top_k(y_pred, 5)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1400, 5) (1400,)
precision 0.060142857142857144
update_recall:  0.3007142857142857
recall 0.3007142857142857
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 421.0, 979.0, 421.0, 6579.0]
TMP_RANK:  TopKV2(values=array([[405, 126, 124, 123, 122],
       [336, 334,   7,   6,   5],
       [379, 325, 324, 322, 296],
       ...,
       [216, 214, 131, 128, 127],
       [131, 130, 129, 128, 127],
       [ 12,  11,  10,   8,   6]]), indices=array([[2, 1, 0, 3, 4],
       [0, 2, 3, 1, 4],
       [4, 0, 2, 1, 3],
       ...,
       [1, 4, 0, 3, 2],
       [4, 1, 3, 2, 0],
       [3, 0, 1, 2, 4]], dtype=int32))


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 10)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 10)

tmp_rank = tf.nn.top_k(y_pred, 10)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1400, 10) (1400,)
precision 0.04235714285714286
update_recall:  0.4235714285714286
recall 0.4235714285714286
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 593.0, 807.0, 593.0, 13407.0]
TMP_RANK:  TopKV2(values=array([[413, 405, 130, ..., 123, 122, 120],
       [336, 334,  62, ...,   7,   6,   5],
       [379, 364, 325, ..., 305, 296, 126],
       ...,
       [218, 216, 214, ..., 128, 127, 124],
       [131, 130, 129, ..., 124, 123, 122],
       [ 30,  28,  12, ...,   6,   4,   2]]), indices=array([[5, 2, 8, ..., 3, 4, 9],
       [0, 2, 7, ..., 3, 1, 4],
       [4, 8, 0, ..., 6, 3, 9],
       ...,
       [8, 1, 4, ..., 3, 2, 7],
       [4, 1, 3, ..., 7, 9, 8],
       [7, 8, 3, ..., 4, 6, 9]], dtype=int32))


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 15)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 15)

tmp_rank = tf.nn.top_k(y_pred, 15)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1400, 15) (1400,)
precision 0.03333333333333333
update_recall:  0.5
recall 0.5
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 700.0, 700.0, 700.0, 20300.0]
TMP_RANK:  TopKV2(values=array([[415, 413, 405, ..., 123, 122, 120],
       [379, 348, 336, ...,   7,   6,   5],
       [394, 379, 364, ..., 296, 127, 126],
       ...,
       [385, 220, 218, ..., 127, 124, 122],
       [133, 132, 131, ..., 120, 118, 117],
       [222, 126,  30, ...,   5,   4,   2]]), indices=array([[12,  5,  2, ...,  3,  4,  9],
       [14, 10,  0, ...,  3,  1,  4],
       [11,  4,  8, ...,  3, 13,  9],
       ...,
       [13, 12,  8, ...,  2,  7, 11],
       [14, 10,  4, ..., 12, 13, 11],
       [13, 14,  7, ..., 10,  6,  9]], dtype=int32))


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 20)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 20)

tmp_rank = tf.math.top_k(y_pred, 20,sorted=False)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1400, 20) (1400,)
precision 0.028035714285714285
update_recall:  0.5607142857142857
recall 0.5607142857142857
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 785.0, 615.0, 785.0, 27215.0]
TMP_RANK:  TopKV2(values=array([[416, 415, 413, ..., 123, 122, 120],
       [379, 348, 339, ...,   7,   6,   5],
       [394, 379, 364, ..., 125, 124, 100],
       ...,
       [394, 385, 231, ..., 125, 124, 122],
       [218, 213, 209, ..., 117, 116, 115],
       [222, 144, 126, ...,   5,   4,   2]]), indices=array([[15, 12,  5, ...,  3,  4,  9],
       [14, 10, 16, ...,  3,  1,  4],
       [11,  4,  8, ..., 18, 19, 16],
       ...,
       [19, 13, 15, ..., 18,  7, 11],
       [18, 17, 19, ..., 11, 16, 15],
       [13, 17, 14, ..., 10,  6,  9]], dtype=int32))


In [ ]:
def mrr_metric(labels, predictions, weights=None,
              metrics_collections=None,
              updates_collections=None,
              name=None):
    
    with tf.name_scope(name, 'mrr_metric', [predictions, labels, weights]) as scope:

    
        k = 5 #predictions.get_shape().as_list()[-1]
        print(predictions.get_shape())

        get_ranked_indicies = tf.expand_dims(tf.where(tf.equal(tf.cast(predictions,tf.int64),labels[:,None]))[:,1],1)
        rr = 1/(get_ranked_indicies+1)
        m_rr =  tf.reduce_sum(rr)/tf.cast(labels.get_shape().as_list()[0],dtype=tf.float64)

        if metrics_collections:
            tf.add_to_collection(metrics_collections, m_rr)

        if updates_collections:
            tf.add_to_collections(updates_collections, update_mrr_op)

        return m_rr,m_rr,rr

In [ ]:
mrr, update_mrr,rr = mrr_metric(y_true,y_pred)
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("update_mrr",sess.run(update_mrr),sess.run(rr).shape)
    # print("precision",sess.run(precision))

    # print("update_recall: ",sess.run(update_recall ))
    # print("recall",sess.run(recall))

    # print("STREAM_VARS: ",(sess.run(stream_vars)))
    # print("TMP_RANK: ",sess.run(tmp_rank))

(1400, 5)
update_mrr 0.3319999999999999 (678, 1)
